In [1]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ref_adata = sc.read('data/czi_8M_PBMC_lataq_ref_10k_loSuHanifa.h5ad')

In [3]:
query_adata = sc.read('data/czi_8M_PBMC_lataq_querySu_10k_loSuHanifa.h5ad')

In [5]:
adata = ref_adata.concatenate(query_adata)

/home/icb/soroor.hediyeh-zadeh/miniconda3/envs/scvi/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


In [6]:
adata

AnnData object with n_obs × n_vars = 8195299 × 10000
    obs: 'dataset', 'sample_ID_czi', 'subject_ID_czi', 'cell_type_ontology_term_id', 'cell_type', 'total_counts_lisa', 'batch_czi', 'assay_ontology_term_id', 'assay', 'development_stage_ontology_term_id', 'development_stage', 'disease_ontology_term_id', 'disease', 'ethnicity_ontology_term_id', 'ethnicity', 'organism_ontology_term_id', 'organism', 'sex_ontology_term_id', 'sex', 'tissue_ontology_term_id', 'tissue', 'cell_type_for_integration', 'sample_ID_lataq', 'batch'
    var: 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'highly_variable_nbatches-0'

In [8]:
adata = adata[~adata.obs.dataset.isin(['villani_pbmc','vento_tormo_pbmc'])]

In [9]:
adata

View of AnnData object with n_obs × n_vars = 8038294 × 10000
    obs: 'dataset', 'sample_ID_czi', 'subject_ID_czi', 'cell_type_ontology_term_id', 'cell_type', 'total_counts_lisa', 'batch_czi', 'assay_ontology_term_id', 'assay', 'development_stage_ontology_term_id', 'development_stage', 'disease_ontology_term_id', 'disease', 'ethnicity_ontology_term_id', 'ethnicity', 'organism_ontology_term_id', 'organism', 'sex_ontology_term_id', 'sex', 'tissue_ontology_term_id', 'tissue', 'cell_type_for_integration', 'sample_ID_lataq', 'batch'
    var: 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'highly_variable_nbatches-0'

In [10]:
mdata = adata.obs[['dataset','sample_ID_czi','subject_ID_czi','batch_czi','assay','sex','disease']]
mdata.drop_duplicates(inplace = True)

/tmp/ipykernel_44630/127768700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata.drop_duplicates(inplace = True)


In [11]:
len(set(adata.obs['sample_ID_lataq'])) # matches above computations 

2503

In [12]:
adata.obs['sample_ID_lataq'] = adata.obs['sample_ID_lataq'].astype('category')

/tmp/ipykernel_44630/1489121291.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['sample_ID_lataq'] = adata.obs['sample_ID_lataq'].astype('category')


In [13]:
dataset_sample = adata.obs[['dataset','sample_ID_lataq']].drop_duplicates(subset = ['sample_ID_lataq'])

In [14]:
dataset_sample.dataset.value_counts()

powell_eqtl                                    981
10_1126_scitranslmed_abh2624                   179
ye_lupus_lupus                                 175
10_1038_s41591_021_01329_2                     143
10_1016_j_cell_2022_01_012                     140
10_1016_j_cell_2021_01_053_convalescence       136
powell_pbmc                                    128
10_1016_j_cell_2021_01_053_nonconvalescence    106
ye_lupus_normal                                 99
10_1038_s41586_021_04345_x                      80
10_1016_j_cell_2021_02_018_adaptive             67
10_1016_j_cell_2021_02_018_innate               67
10_1016_j_isci_2021_102404                      52
10_1016_j_cell_2020_08_001                      49
nygc_pbmc                                       24
10_1126_sciimmunol_abd1554                      20
10_1038_s41591_020_0944_y                       14
10_1126_science_abc6261                         12
10_1038_s41467_020_17834_w                       7
10_1101_2021_07_19_452956_blood

In [15]:
len(set(adata.obs.dataset))

34

In [16]:
print(len(set(mdata.sample_ID_czi)))
print(len(set(mdata.subject_ID_czi)))
print('Number of unique ids')
mdata.shape[0]

2424
1982
Number of unique ids


2503

In [19]:
# make the reference-query split

query_dataset = ['10_1126_sciimmunol_abd1554','10_1038_s41591_021_01329_2']

query_adata = adata[adata.obs.dataset.isin(query_dataset)].copy()
ref_adata = adata[~adata.obs.dataset.isin(query_dataset)].copy()

In [20]:
len(set(query_adata.obs.sample_ID_lataq))

163

In [21]:
len(set(ref_adata.obs.sample_ID_lataq))

2340